# 3 The MAXQ Value Function Decomposition
At the center of the MAXQ method for hierarchical reinforcement learning is the MAXQ value function decomposition. MAXQ describes how to decompose the overall value function for a policy into a collection of value functions for individual subtasks (and subsubtasks, recursively).

## 3.1 A Motivating Example

To make the discussion concrete, let us consider the following simple example. Figure 1 shows a 5-by-5 grid world inhabited by a taxi agent. There are four specially-designated locations in this world, marked as R(ed), B(lue), G(reen), and Y(ellow). The taxi problem is episodic. In each episode, the taxi starts in a randomly-chosen square. There is a passenger at one of the four locations (chosen randomly), and that passenger wishes to be transported to one of the four locations (also chosen randomly). The taxi must go to the passenger’s location (the “source”), pick up the passenger, go to the destination location (the “destination”), and put down the passenger there. (To keep things uniform, the taxi must pick up and drop oﬀ the passenger even if he/she is already located at the destination!) The episode ends when the passenger is deposited at the destination location. There are six primitive actions in this domain: (a) four navigation actions that move the taxi one square North, South, East, or West, (b) a Pickup action, and (c) a Putdown action. Each action is deterministic. There is a reward of −1 for each action and an additional reward of +20 for successfully delivering the passenger. There is a reward of −10 if the taxi attempts to execute the Putdown or Pickup actions illegally. If a navigation action would cause the taxi to hit a wall, the action is a no-op, and there is only the usual reward of −1. We seek a policy that maximizes the total reward per episode. There are 500 possible states: 25 squares, 5 locations for the passenger (counting the four starting locations and the taxi), and 4 destinations. This task has a simple hierarchical structure in which there are two main sub-tasks: Get the passenger and Deliver the passenger. Each of these subtasks in turn involves the subtask of navigating to one of the four locations and then performing a Pickup or Putdown action. This task illustrates the need to support temporal abstraction, state abstraction, and subtask sharing. The temporal abstraction is obvious—for example, the process of navigating to the passenger’s location and picking up the passenger is a temporally extended action that can take diﬀerent numbers of steps to complete depending on the distance to the target. The top level policy (get passenger; deliver passenger) can be expressed very simply if these temporal abstractions can be
9
employed. The need for state abstraction is perhaps less obvious. Consider the subtask of getting the passenger. While this subtask is being solved, the destination of the passenger is completely irrelevant—it cannot aﬀect any of the nagivation or pickup decisions. Perhaps more importantly, when navigating to a target location (either the source or destination location of the passenger), only the identity of the target location is important. The fact that in some cases the taxi is carrying the passenger and in other cases it is not is irrelevant. Finally, support for subtask sharing is critical. If the system could learn how to solve the navigation subtask once, then the solution could be shared by both of the “Get the passenger” and “Deliver the passenger” subtasks. We will show below that the MAXQ method provides a value function representation and learning algorithm that supports temporal abstraction, state abstraction, and subtask sharing. To construct a MAXQ decomposition for the taxi problem, we must identify a set of individual subtasks that we believe will be important for solving the overall task. In this case, let us deﬁne the following four tasks:
• Navigate(t). In this subtask, the goal is to move the taxi from its current location to one of the four target locations, which will be indicated by the formal parameter t.
• Get. In this subtask, the goal is to move the taxi from its current location to the passenger’s current location and pick up the passenger.
• Put. The goal of this subtask is to move the taxi from the current location to the passenger’s destination location and drop oﬀ the passenger.
• Root. This is the whole taxi task.
Each of these subtasks is deﬁned by a subgoal, and each subtask terminates when the subgoal is achieved. After deﬁning these subtasks, we must indicate for each subtask which other subtasks or primitive actions it should employ to reach its goal. For example, the Navigate(t) subtask should use the four primitive actions North, South, East, and West. The Get subtask should use the Navigate subtask and the Pickup primitive action, and so on. All of this information can be summarized by a directed acyclic graph called the task graph, which is shown in Figure 2. In this graph, each node corresponds to a subtask or a primitive action, and each edge corresponds to a potential way in which one subtask can “call” one of its child tasks. The notation formal/actual (e.g., t/source) tells how a formal parameter is to be bound to an actual parameter. Now suppose that for each of these subtasks, we write a policy (e.g., as a computer program) to achieve the subtask. We will refer to the policy for a subtask as a “subroutine”, and we can view the parent subroutine as invoking the child subroutine via ordinary subroutine-call-and-return semantics. If we have a policy for each subtask, then this gives us an overall policy for the Taxi MDP. The Root subtask executes its policy by calling subroutines that are policies for the Get and Put subtasks. The Get policy calls subroutines for the Pickup primitive action and the Navigate(t) subtask. And so on. We will call this collection of policies a hierarchical policy. In a hierarchical policy, each subroutine executes until it enters a terminal state for its subtask.
